In [4]:
!pip install flask-cors

In [5]:
import flask_cors
print("flask_cors is installed!")

flask_cors is installed!


127.0.0.1 - - [15/Mar/2025 10:45:18] "OPTIONS /predict HTTP/1.1" 200 -
127.0.0.1 - - [15/Mar/2025 10:45:20] "OPTIONS /predict HTTP/1.1" 200 -


In [1]:
from flask import Flask, request, jsonify
import joblib
import pandas as pd
import numpy as np
import threading
from flask_cors import CORS

# Load the trained model and scaler
knn = joblib.load('knn_model.pkl')
scaler = joblib.load('scaler.pkl')

# Load the expected feature columns from training
expected_columns = joblib.load('expected_columns.pkl')  # Save this during training

# Define Flask app
app = Flask(__name__)

@app.route('/predict', methods=['POST'])
def predict():
    try:
        data = request.get_json()  # Get JSON input
        df = pd.DataFrame([data])  # Convert input data to DataFrame

        # Apply same transformations as during training
        df = pd.get_dummies(df, drop_first=True)

        # **Fix: Add missing columns efficiently using pd.concat()**
        missing_cols = list(set(expected_columns) - set(df.columns))
        if missing_cols:
            missing_df = pd.DataFrame(0, index=df.index, columns=missing_cols)  # Create missing columns at once
            df = pd.concat([df, missing_df], axis=1)  # Combine

        # Ensure the column order matches the training data
        df = df[expected_columns]

        # Scale input features
        X_scaled = scaler.transform(df)

        # Make prediction
        prediction = knn.predict(X_scaled)

        return jsonify({'prediction': prediction.tolist()})

    except Exception as e:
        return jsonify({'error': str(e)})

# Function to run Flask in a separate thread
def run_flask():
    app.run(port=5000, debug=True, use_reloader=False)

# Start Flask in a background thread
thread = threading.Thread(target=run_flask)
thread.start()

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [15/Mar/2025 10:47:08] "OPTIONS /predict HTTP/1.1" 200 -
127.0.0.1 - - [15/Mar/2025 10:47:11] "OPTIONS /predict HTTP/1.1" 200 -
127.0.0.1 - - [15/Mar/2025 10:51:53] "GET /predict HTTP/1.1" 405 -
127.0.0.1 - - [15/Mar/2025 10:51:53] "GET /favicon.ico HTTP/1.1" 404 -


In [1]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import threading
import joblib
import pandas as pd
import numpy as np
import threading

# Load the trained model and scaler
knn = joblib.load('knn_model.pkl')
scaler = joblib.load('scaler.pkl')

# Load the expected feature columns from training
expected_columns = joblib.load('expected_columns.pkl')

app = Flask(__name__)

# Enable CORS for all routes
CORS(app, resources={r"/predict": {"origins": "*"}})

@app.route('/predict', methods=['POST'])
def predict():
    try:
        data = request.get_json()  # Get JSON input
        df = pd.DataFrame([data])  # Convert input data to DataFrame

        # Apply same transformations as during training
        df = pd.get_dummies(df, drop_first=True)

        # **Fix: Add missing columns efficiently using pd.concat()**
        missing_cols = list(set(expected_columns) - set(df.columns))
        if missing_cols:
            missing_df = pd.DataFrame(0, index=df.index, columns=missing_cols)  # Create missing columns at once
            df = pd.concat([df, missing_df], axis=1)  # Combine

        # Ensure the column order matches the training data
        df = df[expected_columns]

        # Scale input features
        X_scaled = scaler.transform(df)

        # Make prediction
        prediction = knn.predict(X_scaled)

        return jsonify({'predicted_country': prediction.tolist()})

    except Exception as e:
        return jsonify({'error': str(e)})

# Run Flask in a background thread
def run_flask():
    app.run(debug=True, use_reloader=False)

# Start the server only once
threading.Thread(target=run_flask).start()


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [15/Mar/2025 11:16:02] "OPTIONS /predict HTTP/1.1" 200 -
127.0.0.1 - - [15/Mar/2025 11:16:02] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [15/Mar/2025 11:17:00] "OPTIONS /predict HTTP/1.1" 200 -
127.0.0.1 - - [15/Mar/2025 11:17:00] "POST /predict HTTP/1.1" 200 -


In [2]:
import requests

# Sample input data
input_data = {
    "Gender": "Male",
    "Age": 35,
    "Origin_Country": "Thailand",
    "State": "Bangkok",
    "Hotel_Rating": 4,
    "No_of_Days": 5,
    "No_Of_People": 2
}

# Send a POST request to the API
response = requests.post("http://127.0.0.1:5000/predict", json=input_data)

# Print the response
print(response.json())

{'prediction': [0]}


In [3]:
import requests

# Sample input data
input_data = {
    "Gender": "Female",
    "Age": 65,
    "Origin_Country": "Malaysia",
    "State": "Johor",
    "Hotel_Rating": 3,
    "No_of_Days": 25,
    "No_Of_People": 8
}

# Send a POST request to the API
response = requests.post("http://127.0.0.1:5000/predict", json=input_data)

# Print the response
print(response.json())

{'prediction': [3]}
